In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
import re
from wordcloud import WordCloud, STOPWORDS
from textstat import lexicon_count
from textstat import flesch_reading_ease
from textstat import flesch_kincaid_grade
from textstat import sentence_count
from textstat import lexicon_count
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import English
import spacy
from spacy.lang.en.examples import sentences 
import en_core_web_md

2022-04-30 00:41:14.403054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-30 00:41:14.403083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [84]:
# utworzenie ramki danych
all_files = os. listdir('./ecgen-radiology')

d = {
    'COMPARISON' : [],
    'INDICATION' : [],
    'FINDINGS'   : [],
    'IMPRESSION' : [],
    'CAPTION'    : []
}

for f in all_files:
    file = minidom.parse('ecgen-radiology/' + f)
    
    caption = file.getElementsByTagName('caption')
    
    try:
        d['CAPTION'].append(caption[0].childNodes[0].data)
    except IndexError:
        d['CAPTION'].append(None)
    
    models = file.getElementsByTagName('AbstractText')
    
    for i in range(len(models)):
        key = str(models[i].attributes['Label'].value)
        try:
            val = str(models[i].childNodes[0].data)
        except IndexError:
            val = ''
            
        d[key].append(val)
        
for k in d.keys():
    d[k] = np.array(d[k])
    
df = pd.DataFrame(d)

In [85]:
df.head(5)

,COMPARISON,INDICATION,FINDINGS,IMPRESSION,CAPTION
0,"XXXX, XXXX.",Post pacemaker placement.,"Dual-XXXX cardiac pacemaker is in stable, XXXX...","Dual-XXXX cardiac pacemaker in stable, XXXX po...",None
1,"XXXX, XXXX","Dyspnea, leg XXXX, cellulitis.",,Moderate enlargement of the heart XXXX. This s...,"2 VIEW CHEST: XXXX, XXXX at XXXX hours."
2,None,"XXXX-year-old female, XXXX, XXXX, chest pain.",Right lower lobe patchy opacities noted. Heart...,"Patchy right lower lobe opacities, XXXX infect...",PA and lateral views of the chest dated XXXX.
3,None,"Syncope with collapse, XXXX, pain",Heart size and mediastinal contour are normal....,Findings suggestive of emphysema. No acute car...,Xray Chest PA and Lateral
4,None.,"XXXX chest pain, XXXX, history of acid reflux.",The cardiomediastinal silhouette and pulmonary...,1. Clear lungs. 2. Mildly blunted right costop...,"PA and lateral views of the chest dated XXXX, ..."


In [86]:
df = df.drop("COMPARISON", axis = 1)

## Zmiana na małe litery, Usunięcie cyfr, Usunięcie śmieci

In [50]:
def clean(data):
    for c in data.columns:
        data[c] = data[c].apply(lambda x : x.lower())
        data[c] = data[c].apply(lambda x : re.sub(r'[^A-Za-z ]+', ' ', x))
        data[c] = data[c].apply(lambda x : re.sub(r'[A-Za-z]*xxxx[A-Za-z]*', '', x))
    
    return data

In [87]:
df = df.fillna('')
df2 = clean(df)
df2.head()

,INDICATION,FINDINGS,IMPRESSION,CAPTION
0,post pacemaker placement,dual cardiac pacemaker is in stable positio...,dual cardiac pacemaker in stable position ...,
1,dyspnea leg cellulitis,,moderate enlargement of the heart this shows...,view chest at hours
2,year old female chest pain,right lower lobe patchy opacities noted heart...,patchy right lower lobe opacities infectious...,pa and lateral views of the chest dated
3,syncope with collapse pain,heart size and mediastinal contour are normal ...,findings suggestive of emphysema no acute car...,xray chest pa and lateral
4,chest pain history of acid reflux,the cardiomediastinal silhouette and pulmonary...,clear lungs mildly blunted right costophr...,pa and lateral views of the chest dated


## Part of speech tagging

In [88]:
test_sentence = df2['IMPRESSION'][1]
print(test_sentence)

moderate enlargement of the heart   this shows significant interval increase in size from the remote  comparison   cardiomegaly or pericardial fluid could have this appearance  minimal patchy infiltrate atelectasis in medial right base and lingular scarring or subsegmental atelectasis  no pleural effusion or pneumothorax 


In [80]:
tokenized_sent = nltk.sent_tokenize(test_sentence)
[nltk.pos_tag(nltk.word_tokenize(word)) for word in tokenized_sent]

[[('moderate', 'JJ'),
  ('enlargement', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('heart', 'NN'),
  ('this', 'DT'),
  ('shows', 'VBZ'),
  ('significant', 'JJ'),
  ('interval', 'JJ'),
  ('increase', 'NN'),
  ('in', 'IN'),
  ('size', 'NN'),
  ('from', 'IN'),
  ('the', 'DT'),
  ('remote', 'JJ'),
  ('comparison', 'NN'),
  ('cardiomegaly', 'NN'),
  ('or', 'CC'),
  ('pericardial', 'JJ'),
  ('fluid', 'NN'),
  ('could', 'MD'),
  ('have', 'VB'),
  ('this', 'DT'),
  ('appearance', 'NN'),
  ('minimal', 'JJ'),
  ('patchy', 'NN'),
  ('infiltrate', 'NN'),
  ('atelectasis', 'NN'),
  ('in', 'IN'),
  ('medial', 'JJ'),
  ('right', 'JJ'),
  ('base', 'NN'),
  ('and', 'CC'),
  ('lingular', 'JJ'),
  ('scarring', 'NN'),
  ('or', 'CC'),
  ('subsegmental', 'JJ'),
  ('atelectasis', 'NN'),
  ('no', 'DT'),
  ('pleural', 'JJ'),
  ('effusion', 'NN'),
  ('or', 'CC'),
  ('pneumothorax', 'NN')]]

In [104]:
# uprośćmy trochę POS i wyodrębniajmy tylko najważniejsze części mowy

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return 'adjective'
    elif nltk_tag.startswith('V'):
        return 'verb'
    elif nltk_tag.startswith('N'):
        return 'noun'
    elif nltk_tag.startswith('R'):
        return 'adverb'
    elif nltk_tag.startswith('PR'):
        return 'pronoun'
    elif nltk_tag.startswith('W'):
        return 'wh'
    elif nltk_tag.startswith('M'):
        return 'modal' 
    else:         
        return 'None'

In [81]:
pos_tagged = nltk.pos_tag(nltk.word_tokenize(a)) 
pos_easy_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
pos_easy_tagged

[('moderate', 'adjective'),
 ('enlargement', 'noun'),
 ('of', None),
 ('the', None),
 ('heart', 'noun'),
 ('this', None),
 ('shows', 'verb'),
 ('significant', 'adjective'),
 ('interval', 'adjective'),
 ('increase', 'noun'),
 ('in', None),
 ('size', 'noun'),
 ('from', None),
 ('the', None),
 ('remote', 'adjective'),
 ('comparison', 'noun'),
 ('cardiomegaly', 'noun'),
 ('or', None),
 ('pericardial', 'adjective'),
 ('fluid', 'noun'),
 ('could', 'modal'),
 ('have', 'verb'),
 ('this', None),
 ('appearance', 'noun'),
 ('minimal', 'adjective'),
 ('patchy', 'noun'),
 ('infiltrate', 'noun'),
 ('atelectasis', 'noun'),
 ('in', None),
 ('medial', 'adjective'),
 ('right', 'adjective'),
 ('base', 'noun'),
 ('and', None),
 ('lingular', 'adjective'),
 ('scarring', 'noun'),
 ('or', None),
 ('subsegmental', 'adjective'),
 ('atelectasis', 'noun'),
 ('no', None),
 ('pleural', 'adjective'),
 ('effusion', 'noun'),
 ('or', None),
 ('pneumothorax', 'noun')]

In [91]:
def POS(text):
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text)) 
    pos_easy_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    return pos_easy_tagged

In [105]:
df3 = df2.copy(deep=True)

In [106]:
for c in df3.columns:
    df3[c + '_POS'] = ''
    for i in range(len(df3[c])):
        df3[c+'_POS'][i] = POS(df3[c][i])

In [107]:
df3.head(5)

,INDICATION,FINDINGS,IMPRESSION,CAPTION,INDICATION_POS,FINDINGS_POS,IMPRESSION_POS,CAPTION_POS
0,post pacemaker placement,dual cardiac pacemaker is in stable positio...,dual cardiac pacemaker in stable position ...,,"[(post, noun), (pacemaker, noun), (placement, ...","[(dual, adjective), (cardiac, noun), (pacemake...","[(dual, adjective), (cardiac, noun), (pacemake...",[]
1,dyspnea leg cellulitis,,moderate enlargement of the heart this shows...,view chest at hours,"[(dyspnea, noun), (leg, noun), (cellulitis, no...",[],"[(moderate, adjective), (enlargement, noun), (...","[(view, noun), (chest, noun), (at, None), (hou..."
2,year old female chest pain,right lower lobe patchy opacities noted heart...,patchy right lower lobe opacities infectious...,pa and lateral views of the chest dated,"[(year, noun), (old, adjective), (female, noun...","[(right, adverb), (lower, adjective), (lobe, n...","[(patchy, noun), (right, adverb), (lower, adje...","[(pa, noun), (and, None), (lateral, adjective)..."
3,syncope with collapse pain,heart size and mediastinal contour are normal ...,findings suggestive of emphysema no acute car...,xray chest pa and lateral,"[(syncope, noun), (with, None), (collapse, nou...","[(heart, noun), (size, noun), (and, None), (me...","[(findings, noun), (suggestive, noun), (of, No...","[(xray, adjective), (chest, noun), (pa, noun),..."
4,chest pain history of acid reflux,the cardiomediastinal silhouette and pulmonary...,clear lungs mildly blunted right costophr...,pa and lateral views of the chest dated,"[(chest, noun), (pain, noun), (history, noun),...","[(the, None), (cardiomediastinal, adjective), ...","[(clear, adjective), (lungs, noun), (mildly, a...","[(pa, noun), (and, None), (lateral, adjective)..."


In [122]:
def join_POS(l):
    r = []
    for elem in l:
        if len(elem) != 0:
            r.append(elem[1])
    return ' '.join(r)

In [124]:
c = ['INDICATION_POS', 'FINDINGS_POS', 'IMPRESSION_POS', 'CAPTION_POS']

for c in c:
    df3[c + '2'] = ''
    for i in range(len(df3[c])):
        df3[c+'2'][i] = join_POS(df3[c][i])

In [125]:
df3.head(5)

,INDICATION,FINDINGS,IMPRESSION,CAPTION,INDICATION_POS,FINDINGS_POS,IMPRESSION_POS,CAPTION_POS,INDICATION_POS2,INDICATION_POS_POS2,FINDINGS_POS_POS2,IMPRESSION_POS_POS2,CAPTION_POS_POS2
0,post pacemaker placement,dual cardiac pacemaker is in stable positio...,dual cardiac pacemaker in stable position ...,,"[(post, noun), (pacemaker, noun), (placement, ...","[(dual, adjective), (cardiac, noun), (pacemake...","[(dual, adjective), (cardiac, noun), (pacemake...",[],,noun noun noun,adjective noun noun verb None adjective noun N...,adjective noun noun None adjective noun None n...,
1,dyspnea leg cellulitis,,moderate enlargement of the heart this shows...,view chest at hours,"[(dyspnea, noun), (leg, noun), (cellulitis, no...",[],"[(moderate, adjective), (enlargement, noun), (...","[(view, noun), (chest, noun), (at, None), (hou...",,noun noun noun,,adjective noun None None noun None verb adject...,noun noun None noun
2,year old female chest pain,right lower lobe patchy opacities noted heart...,patchy right lower lobe opacities infectious...,pa and lateral views of the chest dated,"[(year, noun), (old, adjective), (female, noun...","[(right, adverb), (lower, adjective), (lobe, n...","[(patchy, noun), (right, adverb), (lower, adje...","[(pa, noun), (and, None), (lateral, adjective)...",,noun adjective noun noun noun,adverb adjective noun noun noun verb noun noun...,noun adverb adjective noun noun adjective noun,noun None adjective noun None None noun verb
3,syncope with collapse pain,heart size and mediastinal contour are normal ...,findings suggestive of emphysema no acute car...,xray chest pa and lateral,"[(syncope, noun), (with, None), (collapse, nou...","[(heart, noun), (size, noun), (and, None), (me...","[(findings, noun), (suggestive, noun), (of, No...","[(xray, adjective), (chest, noun), (pa, noun),...",,noun None noun noun,noun noun None adjective noun verb adjective a...,noun noun None noun None adjective adjective noun,adjective noun noun None adjective
4,chest pain history of acid reflux,the cardiomediastinal silhouette and pulmonary...,clear lungs mildly blunted right costophr...,pa and lateral views of the chest dated,"[(chest, noun), (pain, noun), (history, noun),...","[(the, None), (cardiomediastinal, adjective), ...","[(clear, adjective), (lungs, noun), (mildly, a...","[(pa, noun), (and, None), (lateral, adjective)...",,noun noun noun None noun noun,None adjective noun None adjective noun verb N...,adjective noun adverb verb adjective noun None...,noun None adjective noun None None noun verb


In [126]:
df3['IMPRESSION_POS'][1]

[('moderate', 'adjective'),
 ('enlargement', 'noun'),
 ('of', 'None'),
 ('the', 'None'),
 ('heart', 'noun'),
 ('this', 'None'),
 ('shows', 'verb'),
 ('significant', 'adjective'),
 ('interval', 'adjective'),
 ('increase', 'noun'),
 ('in', 'None'),
 ('size', 'noun'),
 ('from', 'None'),
 ('the', 'None'),
 ('remote', 'adjective'),
 ('comparison', 'noun'),
 ('cardiomegaly', 'noun'),
 ('or', 'None'),
 ('pericardial', 'adjective'),
 ('fluid', 'noun'),
 ('could', 'modal'),
 ('have', 'verb'),
 ('this', 'None'),
 ('appearance', 'noun'),
 ('minimal', 'adjective'),
 ('patchy', 'noun'),
 ('infiltrate', 'noun'),
 ('atelectasis', 'noun'),
 ('in', 'None'),
 ('medial', 'adjective'),
 ('right', 'adjective'),
 ('base', 'noun'),
 ('and', 'None'),
 ('lingular', 'adjective'),
 ('scarring', 'noun'),
 ('or', 'None'),
 ('subsegmental', 'adjective'),
 ('atelectasis', 'noun'),
 ('no', 'None'),
 ('pleural', 'adjective'),
 ('effusion', 'noun'),
 ('or', 'None'),
 ('pneumothorax', 'noun')]

In [133]:
df3['INDICATION_POS'][4]

[('chest', 'noun'),
 ('pain', 'noun'),
 ('history', 'noun'),
 ('of', 'None'),
 ('acid', 'noun'),
 ('reflux', 'noun')]

Tagowanie części mowy radzi sobie dość dobrze nawet z chorobami, w opisach widać wielką dominację rzeczowników.

## Zmiana tekstu na wektory - BERT

BERT jest dobrym wyborem poniważ potrafi rozróżniać takie same słowa o różnym znaczeniu.

In [155]:
df4 = df2[['IMPRESSION']]

In [142]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
#% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [153]:
encoding = tokenizer.encode(test_sentence)
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding))

[101, 8777, 4372, 8017, 20511, 1997, 1996, 2540, 2023, 3065, 3278, 13483, 3623, 1999, 2946, 2013, 1996, 6556, 7831, 4003, 18994, 29107, 2135, 2030, 2566, 5555, 25070, 8331, 2071, 2031, 2023, 3311, 10124, 8983, 2100, 29543, 8823, 2571, 25572, 6190, 1999, 23828, 2157, 2918, 1998, 17002, 7934, 11228, 4892, 2030, 4942, 3366, 21693, 21050, 8823, 2571, 25572, 6190, 2053, 20228, 11236, 2389, 1041, 4246, 14499, 2030, 1052, 2638, 2819, 29288, 2527, 2595, 102]
['[CLS]', 'moderate', 'en', '##lar', '##gement', 'of', 'the', 'heart', 'this', 'shows', 'significant', 'interval', 'increase', 'in', 'size', 'from', 'the', 'remote', 'comparison', 'card', '##iom', '##ega', '##ly', 'or', 'per', '##ica', '##rdial', 'fluid', 'could', 'have', 'this', 'appearance', 'minimal', 'patch', '##y', 'infiltrate', 'ate', '##le', '##cta', '##sis', 'in', 'medial', 'right', 'base', 'and', 'ling', '##ular', 'scar', '##ring', 'or', 'sub', '##se', '##gm', '##ental', 'ate', '##le', '##cta', '##sis', 'no', 'pl', '##eur', '##al'

In [184]:
len(encoding[0])

11

In [166]:
encodings_list = []
for text in df4['IMPRESSION']:
    encoding = tokenizer.encode(text)
    encoding = np.array(encoding).reshape(1, len(encoding))
    encodings_list.append(encoding)

In [189]:
encodings_list

[array([[  101,  7037, 15050,  6393,  8571,  1999,  6540,  2597,  2053,
          3350,  1997, 11325,  4012, 21557,   102]]),
 array([[  101,  8777,  4372,  8017, 20511,  1997,  1996,  2540,  2023,
          3065,  3278, 13483,  3623,  1999,  2946,  2013,  1996,  6556,
          7831,  4003, 18994, 29107,  2135,  2030,  2566,  5555, 25070,
          8331,  2071,  2031,  2023,  3311, 10124,  8983,  2100, 29543,
          8823,  2571, 25572,  6190,  1999, 23828,  2157,  2918,  1998,
         17002,  7934, 11228,  4892,  2030,  4942,  3366, 21693, 21050,
          8823,  2571, 25572,  6190,  2053, 20228, 11236,  2389,  1041,
          4246, 14499,  2030,  1052,  2638,  2819, 29288,  2527,  2595,
           102]]),
 array([[  101,  8983,  2100,  2157,  2896, 21833,  6728,  6305,  6447,
         16514, 29543,   102]]),
 array([[  101,  9556,  6592,  3512,  1997,  7861, 21281,  3366,  2863,
          2053, 11325,  4003,  3695, 14289, 13728,  7856,  2854,  2832,
           102]]),
 array([[  

Sprowadzjąć wektory encodingów do jednakowej długości przez dodawanie zer na końcu można utworzyć z nich macierz.

## Lematyzacja

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

In [45]:
# Tutaj też wykorzystamy Part of speech tagging
# Lematyzacja z WorldNetLemmatizerem i POS tags

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
    
def lemmatize(sentence):
    lemmatizer = WordNetLemmatizer()

    # wykonujemy tokenizacje i lematyzujemy oddzielnie każde słowo w oparciu o to, jaką jest częścią mowy
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence)) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # jeśli słowo nie jest żadnym z 4 wybranych części mowy to nie zmieniamy
            lemmatized_sentence.append(word)
        else:       
            # w przeciwnym wypadku lematyzujemy
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)

    return lemmatized_sentence

In [52]:
# lemmatyzacja ramki danych

df5 = df2.copy(deep=True)

for c in df5.columns:
    df5[c + '_lem'] = ''
    for i in range(len(df5[c])):
        df5[c+'_lem'][i] = lemmatize(df5[c][i])

In [53]:
df5.head(5)

,INDICATION,FINDINGS,IMPRESSION,CAPTION,INDICATION_lem,FINDINGS_lem,IMPRESSION_lem,CAPTION_lem
0,post pacemaker placement,dual cardiac pacemaker is in stable positio...,dual cardiac pacemaker in stable position ...,,post pacemaker placement,dual cardiac pacemaker be in stable position t...,dual cardiac pacemaker in stable position no e...,
1,dyspnea leg cellulitis,,moderate enlargement of the heart this shows...,view chest at hours,dyspnea leg cellulitis,,moderate enlargement of the heart this show si...,view chest at hour
2,year old female chest pain,right lower lobe patchy opacities noted heart...,patchy right lower lobe opacities infectious...,pa and lateral views of the chest dated,year old female chest pain,right low lobe patchy opacity note heart size ...,patchy right low lobe opacity infectious infil...,pa and lateral view of the chest date
3,syncope with collapse pain,heart size and mediastinal contour are normal ...,findings suggestive of emphysema no acute car...,xray chest pa and lateral,syncope with collapse pain,heart size and mediastinal contour be normal p...,finding suggestive of emphysema no acute cardi...,xray chest pa and lateral
4,chest pain history of acid reflux,the cardiomediastinal silhouette and pulmonary...,clear lungs mildly blunted right costophr...,pa and lateral views of the chest dated,chest pain history of acid reflux,the cardiomediastinal silhouette and pulmonary...,clear lung mildly blunt right costophrenic wit...,pa and lateral view of the chest date
